# Build123d 基本動作確認ノートブック

このノートブックでは、Build123dのインストールと基本的な3Dモデル生成を確認します。

**流れ**
1. インストール
2. 基本形状の生成（Box, Cylinder, Sphere）
3. ブーリアン演算（足し算・引き算）
4. フィレット・面取り
5. 機械部品の例（フランジ付きシャフト）
6. ファイル出力（STEP / STL）
7. LLMプロンプトテンプレート

---
> **LLMを使って3Dモデルを生成したい場合は、セル7のプロンプトをコピーして使ってください。**

## 1. インストール

In [ ]:
# build123dのインストール（初回のみ）
import sys
%pip install -q build123d

# Colabの場合、インストール後にランタイム再起動が必要
try:
    from google.colab import runtime
    print('✅ build123d インストール完了')
    print('⚠️  ランタイムを再起動してください（ランタイム → ランタイムを再起動）')
    print('   再起動後はこのセル(セル1)をスキップして、セル2から実行してください。')
except ImportError:
    print('✅ build123d インストール完了（ローカル環境）')


In [ ]:
# インポートと動作確認
from build123d import *
import os

print("✅ build123d インポート成功")
print(f"   バージョン確認: Box(10,10,10) → {Box(10,10,10)}")

## 2. 基本形状の生成

In [ ]:
# ---- Box（直方体）----
with BuildPart() as box_part:
    Box(100, 50, 30)  # 幅100mm, 奥行50mm, 高さ30mm

print(f"✅ Box 生成完了")
print(f"   体積: {box_part.part.volume:.1f} mm³")
print(f"   バウンディングボックス: {box_part.part.bounding_box()}")

# ---- Cylinder（円柱）----
with BuildPart() as cyl_part:
    Cylinder(radius=25, height=80)  # 半径25mm, 高さ80mm

print(f"\n✅ Cylinder 生成完了")
print(f"   体積: {cyl_part.part.volume:.1f} mm³")

# ---- Sphere（球）----
with BuildPart() as sphere_part:
    Sphere(radius=20)  # 半径20mm

print(f"\n✅ Sphere 生成完了")
print(f"   体積: {sphere_part.part.volume:.1f} mm³")

## 3. ブーリアン演算（足し算・引き算）

In [ ]:
# ---- 足し算（Union）: BoxにCylinderを合体 ----
with BuildPart() as union_part:
    Box(60, 60, 20)
    Cylinder(radius=15, height=40)  # Boxの上にCylinderが生える

print(f"✅ Union（合体）生成完了")
print(f"   体積: {union_part.part.volume:.1f} mm³")

In [ ]:
# ---- 引き算（Subtract）: Boxに穴を開ける ----
with BuildPart() as hole_part:
    Box(80, 80, 30)
    with Locations((0, 0, 0)):
        Cylinder(radius=20, height=30, mode=Mode.SUBTRACT)  # 中心に穴

print(f"✅ Subtract（穴あき）生成完了")
print(f"   体積: {hole_part.part.volume:.1f} mm³")

# 参考：穴なしBoxの体積
vol_box = 80 * 80 * 30
vol_hole = 3.14159 * 20**2 * 30
print(f"   検算: Box体積 - 穴体積 ≈ {vol_box - vol_hole:.1f} mm³")

In [ ]:
# ---- 複数の穴：ボルト穴パターン ----
with BuildPart() as bolt_plate:
    Box(100, 100, 15)
    # 四隅にボルト穴（直径10mm = 半径5mm）
    with GridLocations(70, 70, 2, 2):  # 70mmピッチ、2×2グリッド
        Cylinder(radius=5, height=15, mode=Mode.SUBTRACT)

print(f"✅ ボルト穴プレート生成完了")
print(f"   体積: {bolt_plate.part.volume:.1f} mm³")

## 4. フィレット・面取り

In [ ]:
# ---- フィレット（角丸め）----
with BuildPart() as fillet_part:
    Box(80, 60, 25)
    fillet(fillet_part.edges(), radius=5)  # すべてのエッジをR5でフィレット

print(f"✅ フィレット生成完了")
print(f"   体積: {fillet_part.part.volume:.1f} mm³")

# ---- 面取り（Chamfer）----
with BuildPart() as chamfer_part:
    Box(80, 60, 25)
    chamfer(chamfer_part.edges(), length=3)  # すべてのエッジをC3で面取り

print(f"\n✅ 面取り生成完了")
print(f"   体積: {chamfer_part.part.volume:.1f} mm³")

## 5. 機械部品の例：フランジ付きシャフト

LLMが生成するコードの典型的な例です。

In [ ]:
# ---- フランジ付きシャフト ----
# パラメータ（LLMがここを変えて返す）
shaft_diameter   = 30    # シャフト直径 [mm]
shaft_length     = 120   # シャフト長さ [mm]
flange_diameter  = 80    # フランジ直径 [mm]
flange_thickness = 15    # フランジ厚さ [mm]
bolt_hole_dia    = 10    # ボルト穴直径 [mm]
bolt_pcd         = 60    # ボルト穴PCD [mm]
n_bolts          = 4     # ボルト穴数
fillet_r         = 3     # フィレット半径 [mm]

with BuildPart() as flange_shaft:
    # シャフト部
    Cylinder(radius=shaft_diameter/2, height=shaft_length)
    
    # フランジ部（シャフト底面に合体）
    with Locations((0, 0, -shaft_length/2 + flange_thickness/2)):
        Cylinder(radius=flange_diameter/2, height=flange_thickness)
    
    # ボルト穴（フランジに均等配置）
    with PolarLocations(bolt_pcd/2, n_bolts):
        Cylinder(radius=bolt_hole_dia/2, height=flange_thickness, mode=Mode.SUBTRACT)
    
    # シャフトとフランジの境界をフィレット
    fillet(
        flange_shaft.edges().filter_by_position(Axis.Z, -shaft_length/2 + flange_thickness, -shaft_length/2 + flange_thickness + 1),
        radius=fillet_r
    )

print(f"✅ フランジ付きシャフト生成完了")
print(f"   体積: {flange_shaft.part.volume:.1f} mm³")
print(f"   シャフト径: {shaft_diameter}mm, 長さ: {shaft_length}mm")
print(f"   フランジ径: {flange_diameter}mm, ボルト穴: {n_bolts}×φ{bolt_hole_dia}mm")

In [ ]:
# ---- もう一例：Lブラケット ----
width      = 60   # 幅 [mm]
height     = 80   # 高さ [mm]
depth      = 50   # 奥行き [mm]
thickness  = 8    # 板厚 [mm]

with BuildPart() as l_bracket:
    # 縦板
    Box(thickness, width, height)
    # 横板
    with Locations((depth/2, 0, -height/2 + thickness/2)):
        Box(depth, width, thickness)
    # エッジのフィレット
    fillet(l_bracket.edges().filter_by_position(Axis.X, 0, 1), radius=5)

print(f"✅ Lブラケット生成完了")
print(f"   体積: {l_bracket.part.volume:.1f} mm³")

## 6. ファイル出力（STEP / STL）

- **STEP**: FreeCADやSolidWorksで開ける汎用形式
- **STL**: 3Dプリントや後続のメッシャーへ渡す形式

In [ ]:
# 出力ディレクトリの作成
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# フランジ付きシャフトを出力
step_path = f"{OUTPUT_DIR}/flange_shaft.step"
stl_path  = f"{OUTPUT_DIR}/flange_shaft.stl"

export_step(flange_shaft.part, step_path)
export_stl(flange_shaft.part, stl_path)

print(f"✅ STEP出力: {step_path}  ({os.path.getsize(step_path):,} bytes)")
print(f"✅ STL出力:  {stl_path}  ({os.path.getsize(stl_path):,} bytes)")

# Lブラケットも出力
export_step(l_bracket.part, f"{OUTPUT_DIR}/l_bracket.step")
export_stl(l_bracket.part,  f"{OUTPUT_DIR}/l_bracket.stl")
print(f"✅ Lブラケット STEP/STL出力完了")

print(f"\n📁 出力ファイル一覧:")
for f in os.listdir(OUTPUT_DIR):
    print(f"   {f}")

In [ ]:
# Colabからファイルをダウンロードする場合
try:
    from google.colab import files
    files.download(step_path)
    files.download(stl_path)
    print("✅ ダウンロード開始")
except ImportError:
    print("ℹ️ ローカル環境では上記のoutputフォルダを確認してください")

## 7. LLM × CAD ダッシュボード

下の **セットアップセル** を実行すると、GUIパネルが表示されます。
APIキーを持っていない場合は Manual タブのプロンプトをコピーして外部LLMへ貼り付けてください。


In [ ]:
# ── 依存ライブラリ ─────────────────────────────────────────
import sys, os

# Colab環境では google-genai が ipython を上書きして再起動が必要になるため
# インストール前後でバージョンを比較し、必要な場合だけ再起動を促す
def _install_and_maybe_restart():
    import subprocess, importlib

    pkgs = {
        'ipywidgets': 'ipywidgets',
        'anthropic':  'anthropic',
        'openai':     'openai',
        'google.genai': 'google-genai',
    }
    needs_restart = False
    for mod, pkg in pkgs.items():
        try:
            importlib.import_module(mod)
        except ImportError:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', pkg], check=True)
            needs_restart = True

    # google-genai は ipython を更新するので Colab では再起動が必要
    if needs_restart:
        try:
            from google.colab import runtime  # Colab 環境のみ存在
            print('🔄 ライブラリをインストールしました。ランタイムを再起動します...')
            print('   再起動後、このセルをもう一度実行してください。')
            runtime.unassign()
        except ImportError:
            print('✅ ライブラリインストール完了（ローカル環境）')

_install_and_maybe_restart()

import ast, re, textwrap, traceback, os
import ipywidgets as w
from IPython.display import display, HTML, clear_output

os.makedirs('output', exist_ok=True)

# ── 状態 ────────────────────────────────────────────────────
state = dict(
    llm_mode        = 'manual',
    provider        = 'anthropic',   # anthropic / openai / google
    anthropic_key   = '',
    openai_key      = '',
    google_key      = '',
    anthropic_model = 'claude-opus-4-6',
    openai_model    = 'gpt-4o',
    google_model    = 'gemini-2.5-flash',
    history         = [],
    last_code       = '',
    last_err        = '',
    last_raw        = '',
)

SYSTEM_PROMPT = textwrap.dedent("""\
    あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

    【ルール】
    1. 必ず `from build123d import *` でインポートする
    2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
    3. 寸法はすべてmm単位のfloatまたはintで書く
    4. パラメータは変数として冒頭にまとめて定義する
    5. ブーリアン演算は mode=Mode.SUBTRACT（穴）または mode=Mode.ADD（合体）を使う
    6. BuildSketch はネストせず、mode=Mode.SUBTRACT で内穴を作る
    7. fillet はすべての形状確定後、最後に適用する
    8. 最後に必ずこの2行を含める:
       export_step(part.part, "output/llm_output.step")
       export_stl(part.part, "output/llm_output.stl")
    9. 【最重要】必ずコードブロック(```python\n...\n```)のみ返す。
       日本語の説明・コメントをコードブロックの外に書かない。
       返答の最初の文字は ``` でなければならない。

    【基本形状】Box / Cylinder / Sphere / Cone
    【配置】Locations / PolarLocations / GridLocations

    【fillet / chamfer の正しい書き方 ─ 最重要】
    # ✅ 正しい: グローバル関数として呼び出す
    with BuildPart() as part:
        Box(100, 100, 30)
        fillet(part.edges(), radius=5)        # part.edges() を渡す
        chamfer(part.edges(), length=3)       # chamfer も同様

    # ❌ 誤り（AttributeError になる）
    # part.fillet(...)   ← BuildPart オブジェクトにはfilletメソッドはない
    # part.part.fillet(...) ← これも不可

    【よくある間違い】
    - BuildSketchのネスト禁止
    - fillet/chamfer は必ず形状確定後にグローバル関数で呼ぶ
    - part.edges() の代わりに part.part.edges() と書かない
    - filter_by_orientation / filter_by_axis は存在しない → 使わない
    - 特定エッジだけにfilletしたい場合は filter_by_position(Axis.Z, min, max) を使う
      例: fillet(part.edges().filter_by_position(Axis.Z, 0, height), radius=3)
""")

BANNED = ['os.system','subprocess','eval(','exec(','__import__','shutil.','requests.','urllib']

def extract_code(text):
    """LLMの返答からPythonコードブロックを確実に抽出する"""
    # パターン1: ```python ... ``` または ``` ... ```
    m = re.search(r'```(?:python)?[ \t]*\n(.*?)```', text, re.DOTALL)
    if m:
        return m.group(1).strip()
    # パターン2: バッククォートなしでも from build123d import で始まる行以降を取得
    m = re.search(r'(from build123d import.*)', text, re.DOTALL)
    if m:
        return m.group(1).strip()
    # パターン3: どうしても見つからない場合は空文字を返してエラーにする
    return ''

def validate_code_block(code, raw_response):
    """コードが空または明らかに不正な場合にエラーメッセージを返す"""
    if not code:
        return False, (
            'LLMがコードブロックを返しませんでした。\n'
            f'LLMの返答（先頭200文字）:\n{raw_response[:200]}'
        )
    if 'from build123d' not in code and 'BuildPart' not in code:
        return False, (
            'Build123dのコードが含まれていません。\n'
            f'抽出されたテキスト（先頭200文字）:\n{code[:200]}'
        )
    return True, ''

def safety_check(code):
    return [b for b in BANNED if b in code]

def auto_patch(code):
    """LLMがよく間違えるパターンを実行前に自動修正する"""
    import re
    patches = []

    # パターン1: part.fillet(...) → fillet(part.edges(), ...)
    # 変数名が何であっても対応（part, result, body など）
    def fix_method_fillet(m):
        varname = m.group(1)
        args = m.group(2)
        patches.append(f'  {varname}.fillet({args}) → fillet({varname}.edges(), {args})')
        return f'fillet({varname}.edges(), {args})'
    code = re.sub(r'(\w+)\.fillet\(([^)]+)\)', fix_method_fillet, code)

    # パターン2: part.chamfer(...) → chamfer(part.edges(), ...)
    def fix_method_chamfer(m):
        varname = m.group(1)
        args = m.group(2)
        patches.append(f'  {varname}.chamfer({args}) → chamfer({varname}.edges(), {args})')
        return f'chamfer({varname}.edges(), {args})'
    code = re.sub(r'(\w+)\.chamfer\(([^)]+)\)', fix_method_chamfer, code)

    # パターン3: fillet(part.part.edges(), ...) → fillet(part.edges(), ...)
    code = re.sub(r'fillet\((\w+)\.part\.edges\(\)', r'fillet(\1.edges()', code)
    code = re.sub(r'chamfer\((\w+)\.part\.edges\(\)', r'chamfer(\1.edges()', code)

    # パターン4: .filter_by_orientation(...) → 削除（存在しないメソッド）
    if '.filter_by_orientation' in code:
        code = re.sub(r'\.filter_by_orientation\([^)]*\)', '', code)
        patches.append('  .filter_by_orientation(...) → 削除（存在しないメソッド）')

    # パターン5: .filter_by_axis(...) → 削除（存在しないメソッド）
    if '.filter_by_axis' in code:
        code = re.sub(r'\.filter_by_axis\([^)]*\)', '', code)
        patches.append('  .filter_by_axis(...) → 削除（存在しないメソッド）')

    # パターン6: .filter_by_type(...) → 削除
    if '.filter_by_type' in code:
        code = re.sub(r'\.filter_by_type\([^)]*\)', '', code)
        patches.append('  .filter_by_type(...) → 削除（すべてのエッジ対象に変更）')

    return code, patches

def run_code(code):
    # 自動パッチ適用
    code, patches = auto_patch(code)
    if patches:
        print('🔧 自動修正を適用しました:')
        for p in patches:
            print(p)

    dangers = safety_check(code)
    if dangers:
        return False, f'安全チェックNG: {dangers}'
    try:
        ast.parse(code)
    except SyntaxError as e:
        return False, f'構文エラー: {e}'
    try:
        exec(compile(code, '<llm_generated>', 'exec'), {'__builtins__': __builtins__})
        return True, ''
    except Exception:
        return False, traceback.format_exc()

def call_api(user_msg, history):
    msgs = list(history) + [{'role': 'user', 'content': user_msg}]
    p = state['provider']

    if p == 'anthropic':
        import anthropic
        c = anthropic.Anthropic(api_key=state['anthropic_key'])
        r = c.messages.create(model=state['anthropic_model'],
                               max_tokens=4096, system=SYSTEM_PROMPT, messages=msgs)
        return r.content[0].text

    elif p == 'openai':
        from openai import OpenAI
        c = OpenAI(api_key=state['openai_key'])
        r = c.chat.completions.create(
            model=state['openai_model'],
            messages=[{'role':'system','content':SYSTEM_PROMPT}] + msgs,
            max_tokens=4096)
        return r.choices[0].message.content

    elif p == 'google':
        import time
        from google import genai
        from google.genai import types
        from google.genai.errors import APIError
        c = genai.Client(api_key=state['google_key'])
        gc_msgs = []
        for m in msgs[:-1]:
            role = 'user' if m['role'] == 'user' else 'model'
            gc_msgs.append(types.Content(role=role, parts=[types.Part(text=m['content'])]))
        contents = gc_msgs + [types.Content(role='user', parts=[types.Part(text=user_msg)])]
        # 429レート制限の場合は待機してリトライ
        for wait in [0, 15, 30]:
            if wait:
                time.sleep(wait)
            try:
                r = c.models.generate_content(
                    model=state['google_model'],
                    contents=contents,
                    config=types.GenerateContentConfig(
                        system_instruction=SYSTEM_PROMPT,
                        max_output_tokens=4096))
                return r.text
            except APIError as e:
                if e.code == 429 and wait < 30:
                    continue   # リトライ
                raise
    else:
        raise ValueError('プロバイダーが未設定です')

def test_connection(provider):
    """APIキーの疎通確認（最小リクエスト）"""
    try:
        if provider == 'anthropic':
            import anthropic
            c = anthropic.Anthropic(api_key=state['anthropic_key'])
            c.messages.create(model=state['anthropic_model'],
                               max_tokens=8, messages=[{'role':'user','content':'hi'}])
        elif provider == 'openai':
            from openai import OpenAI
            c = OpenAI(api_key=state['openai_key'])
            c.chat.completions.create(model=state['openai_model'],
                                       messages=[{'role':'user','content':'hi'}], max_tokens=8)
        elif provider == 'google':
            from google import genai
            from google.genai import types
            c = genai.Client(api_key=state['google_key'])
            c.models.generate_content(
                model=state['google_model'],
                contents='hi',
                config=types.GenerateContentConfig(max_output_tokens=8))
        return True, '接続OK'
    except Exception as e:
        return False, str(e)[:120]

# ══════════════════════════════════════════════════════════════
# スタイル
# ══════════════════════════════════════════════════════════════
display(HTML("""
<style>
.cad-log  { background:#0f1117; color:#c8ffc8; padding:12px;
            border-radius:6px; font-size:12px; white-space:pre-wrap;
            max-height:240px; overflow-y:auto; min-height:48px; }
.cad-code { background:#1e1e2e; color:#cdd6f4; padding:12px;
            border-radius:6px; font-size:11px; white-space:pre-wrap;
            max-height:300px; overflow-y:auto; }
.cad-tip  { color:#888; font-size:11px; margin-top:4px; }
.st-ok    { color:#16a34a; font-weight:600; }
.st-ng    { color:#dc2626; font-weight:600; }
.st-idle  { color:#6b7280; }
</style>
<link href='https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@400;600&display=swap' rel='stylesheet'>
"""))

# ══════════════════════════════════════════════════════════════
# ⚙️ 設定タブ
# ══════════════════════════════════════════════════════════════
mode_toggle = w.ToggleButtons(
    options=[('🔑 API モード','api'),('📋 Manual モード','manual')],
    value='manual', description='動作モード:',
    style={'button_width':'150px','description_width':'80px'})

provider_toggle = w.ToggleButtons(
    options=[('🟣 Anthropic','anthropic'),('🟢 OpenAI','openai'),('🔵 Google','google')],
    value='anthropic', description='プロバイダー:',
    style={'button_width':'130px','description_width':'90px'})

# ── Anthropic ──────────────────────────────────────────────
ant_key   = w.Password(placeholder='sk-ant-api03-...', description='APIキー:',
                        style={'description_width':'70px'}, layout=w.Layout(width='460px'))
ant_model = w.Dropdown(options=['claude-opus-4-6','claude-sonnet-4-6','claude-haiku-4-5-20251001'],
                        value='claude-opus-4-6', description='モデル:',
                        style={'description_width':'70px'}, layout=w.Layout(width='360px'))
ant_test  = w.Button(description='🔌 接続テスト', layout=w.Layout(width='130px'))
ant_stat  = w.HTML('<span class="st-idle">未テスト</span>')
ant_box   = w.VBox([
    w.HTML('<b style="font-size:13px">🟣 Anthropic Claude</b>'),
    w.HTML('<span class="cad-tip">APIキーは <a href="https://console.anthropic.com/" target="_blank">console.anthropic.com</a> で取得できます</span>'),
    ant_key, ant_model,
    w.HBox([ant_test, ant_stat], layout=w.Layout(align_items='center', gap='10px')),
], layout=w.Layout(padding='8px 4px'))

# ── OpenAI ─────────────────────────────────────────────────
oai_key   = w.Password(placeholder='sk-...', description='APIキー:',
                        style={'description_width':'70px'}, layout=w.Layout(width='460px'))
oai_model = w.Dropdown(options=['gpt-4o','gpt-4o-mini','gpt-4-turbo'],
                        value='gpt-4o', description='モデル:',
                        style={'description_width':'70px'}, layout=w.Layout(width='360px'))
oai_test  = w.Button(description='🔌 接続テスト', layout=w.Layout(width='130px'))
oai_stat  = w.HTML('<span class="st-idle">未テスト</span>')
oai_box   = w.VBox([
    w.HTML('<b style="font-size:13px">🟢 OpenAI</b>'),
    w.HTML('<span class="cad-tip">APIキーは <a href="https://platform.openai.com/api-keys" target="_blank">platform.openai.com</a> で取得できます</span>'),
    oai_key, oai_model,
    w.HBox([oai_test, oai_stat], layout=w.Layout(align_items='center', gap='10px')),
], layout=w.Layout(padding='8px 4px'))

# ── Google AI Studio ───────────────────────────────────────
goo_key   = w.Password(placeholder='AIza...', description='APIキー:',
                        style={'description_width':'70px'}, layout=w.Layout(width='460px'))
goo_model = w.Dropdown(options=['gemini-2.5-flash','gemini-2.5-flash-lite','gemini-2.0-flash','gemini-2.0-flash-lite'],
                        value='gemini-2.5-flash', description='モデル:',
                        style={'description_width':'70px'}, layout=w.Layout(width='360px'))
goo_test  = w.Button(description='🔌 接続テスト', layout=w.Layout(width='130px'))
goo_stat  = w.HTML('<span class="st-idle">未テスト</span>')
goo_box   = w.VBox([
    w.HTML('<b style="font-size:13px">🔵 Google AI Studio (Gemini)</b>'),
    w.HTML('<span class="cad-tip">APIキーは <a href="https://aistudio.google.com/app/apikey" target="_blank">aistudio.google.com</a> で取得できます（無料枠あり）</span>'),
    goo_key, goo_model,
    w.HBox([goo_test, goo_stat], layout=w.Layout(align_items='center', gap='10px')),
], layout=w.Layout(padding='8px 4px'))

# プロバイダー切り替えで表示を変える
provider_area = w.VBox([ant_box])

def on_provider_change(change=None):
    p = provider_toggle.value
    state['provider'] = p
    if p == 'anthropic':
        provider_area.children = [ant_box]
    elif p == 'openai':
        provider_area.children = [oai_box]
    else:
        provider_area.children = [goo_box]

provider_toggle.observe(on_provider_change, names='value')

def on_mode_change(change=None):
    state['llm_mode'] = mode_toggle.value
    api_only_area.layout.display = 'block' if mode_toggle.value == 'api' else 'none'

api_only_area = w.VBox([
    w.HTML('<hr style="margin:8px 0">'),
    w.HTML('<b>プロバイダー選択</b>'),
    provider_toggle,
    provider_area,
], layout=w.Layout(display='none'))

mode_toggle.observe(on_mode_change, names='value')

settings_tab = w.VBox([
    w.HTML('<b style="font-size:14px">⚙️ 動作モード</b>'),
    mode_toggle,
    api_only_area,
], layout=w.Layout(padding='10px'))

# ── 接続テストのロジック ───────────────────────────────────
def make_test_handler(provider, key_widget, model_widget, stat_widget, test_btn):
    def handler(btn):
        state[provider + '_key']   = key_widget.value
        state[provider + '_model'] = model_widget.value
        state['provider'] = provider
        if not key_widget.value.strip():
            stat_widget.value = '<span class="st-ng">⚠️ キーを入力してください</span>'
            return
        test_btn.disabled = True
        stat_widget.value = '<span class="st-idle">テスト中...</span>'
        ok, msg = test_connection(provider)
        if ok:
            stat_widget.value = f'<span class="st-ok">✅ {msg}</span>'
        else:
            stat_widget.value = f'<span class="st-ng">❌ {msg}</span>'
        test_btn.disabled = False
    return handler

ant_test.on_click(make_test_handler('anthropic', ant_key, ant_model, ant_stat, ant_test))
oai_test.on_click(make_test_handler('openai',    oai_key, oai_model, oai_stat, oai_test))
goo_test.on_click(make_test_handler('google',    goo_key, goo_model, goo_stat, goo_test))

def sync_keys(change=None):
    state['anthropic_key']   = ant_key.value
    state['openai_key']      = oai_key.value
    state['google_key']      = goo_key.value
    state['anthropic_model'] = ant_model.value
    state['openai_model']    = oai_model.value
    state['google_model']    = goo_model.value

for widget in [ant_key, oai_key, goo_key, ant_model, oai_model, goo_model]:
    widget.observe(sync_keys, names='value')

# ══════════════════════════════════════════════════════════════
# 🤖 API 自動生成タブ
# ══════════════════════════════════════════════════════════════
request_box   = w.Textarea(placeholder='例: 外径60mm、内径40mm、長さ150mmのパイプ。両端にR3フィレットあり。',
                            layout=w.Layout(width='98%', height='88px'))
generate_btn  = w.Button(description='🚀 生成', button_style='primary', layout=w.Layout(width='110px'))
retry_btn     = w.Button(description='🔄 リトライ', button_style='danger',
                          layout=w.Layout(width='110px'), disabled=True)
clear_btn     = w.Button(description='🗑️ 履歴クリア', button_style='warning', layout=w.Layout(width='120px'))
max_retry_box = w.BoundedIntText(value=3, min=1, max=10, description='最大回数:',
                                  style={'description_width':'70px'}, layout=w.Layout(width='150px'))
log_out  = w.Output()
code_out = w.Output()

def log(msg):
    with log_out:
        print(msg)

def show_code(code):
    code_out.clear_output()
    with code_out:
        display(HTML(f'<div class="cad-code">{code}</div>'))

def do_generate(btn):
    sync_keys()
    state['llm_mode'] = mode_toggle.value
    req = request_box.value.strip()
    if not req:
        log('⚠️ リクエストを入力してください'); return
    if not (state['anthropic_key'] or state['openai_key'] or state['google_key']):
        log('⚠️ 設定タブでAPIキーを入力してください'); return
    generate_btn.disabled = True
    retry_btn.disabled = True
    log_out.clear_output()
    log('🤖 LLMにリクエスト送信中...')
    try:
        raw = call_api(req, state['history'])
        state['last_raw'] = raw
        code = extract_code(raw)
        valid, verr = validate_code_block(code, raw)
        if not valid:
            log(f'⚠️ コード抽出失敗:\n{verr}')
            log('💡 リトライします...')
            # 抽出失敗時は即リトライリクエストを送る
            fix_req = ('コードブロック(```python ... ```)のみを返してください。説明文は不要です。\n'
                       '前回の返答にはコードブロックが含まれていませんでした。\n'
                       f'元のリクエスト: {req}')
            raw = call_api(fix_req, [])
            code = extract_code(raw)
            valid, verr = validate_code_block(code, raw)
            if not valid:
                log(f'❌ 再試行後もコード抽出失敗:\n{verr}')
                return
        state['last_code'] = code
        show_code(code)
        ok, err = run_code(code)
        state['last_err'] = err
        if ok:
            log('✅ 実行成功！ output/llm_output.step / .stl を確認してください')
            state['history'] += [{'role':'user','content':req},{'role':'assistant','content':raw}]
        else:
            log(f'❌ 実行エラー:\n{err}')
            log('💡 「リトライ」ボタンで自動修正を試みます')
            retry_btn.disabled = False
        refresh_history()
    except Exception as e:
        log(f'⛔ API呼び出しエラー: {e}')
    finally:
        generate_btn.disabled = False

def do_retry(btn):
    sync_keys()
    if not state['last_code']:
        log('ℹ️ 先に生成を実行してください'); return
    retry_btn.disabled = True
    generate_btn.disabled = True
    cur_code = state['last_code']
    cur_err  = state['last_err']
    hist = list(state['history'])
    for n in range(1, max_retry_box.value + 1):
        log(f'🔄 リトライ {n}/{max_retry_box.value} ...')
        fix = (f'以下のコードでエラーが発生しました。修正したコードを返してください。\n\n'
               f'【エラー】\n{cur_err}\n\n【コード】\n```python\n{cur_code}\n```')
        try:
            raw = call_api(fix, hist)
        except Exception as e:
            log(f'⛔ API エラー: {e}'); break
        cur_code = extract_code(raw)
        show_code(cur_code)
        ok, cur_err = run_code(cur_code)
        if ok:
            log(f'✅ リトライ {n} 回目で成功！')
            state['last_code'] = cur_code
            state['last_err']  = ''
            retry_btn.disabled = True
            break
        log(f'❌ まだエラー ({n}回目)')
        hist += [{'role':'user','content':fix},{'role':'assistant','content':raw}]
    else:
        log(f'⛔ {max_retry_box.value} 回試みましたが修正できませんでした')
    generate_btn.disabled = False

def do_clear(btn):
    state['history'] = []; state['last_code'] = ''; state['last_err'] = ''
    log_out.clear_output(); code_out.clear_output()
    log('🗑️ 履歴をクリアしました')
    refresh_history()

generate_btn.on_click(do_generate)
retry_btn.on_click(do_retry)
clear_btn.on_click(do_clear)

api_tab = w.VBox([
    w.HTML('<b>作りたいCADモデルを日本語で入力してください</b>'),
    request_box,
    w.HBox([generate_btn, retry_btn, clear_btn, max_retry_box],
            layout=w.Layout(gap='8px', align_items='center')),
    w.HTML('<b style="margin-top:6px">ログ</b>'),
    log_out,
    w.HTML('<b>生成コード</b>'),
    code_out,
])

# ══════════════════════════════════════════════════════════════
# 📋 Manual タブ
# ══════════════════════════════════════════════════════════════
man_req_box    = w.Textarea(placeholder='例: 外径60mm、内径40mm、長さ150mmのパイプ。両端にR3フィレットあり。',
                             layout=w.Layout(width='98%', height='88px'))
gen_prompt_btn = w.Button(description='📋 プロンプト生成', button_style='info', layout=w.Layout(width='150px'))
prompt_out     = w.Output()
paste_box      = w.Textarea(placeholder='LLMから返ってきたコードをここに貼り付けてください...',
                             layout=w.Layout(width='98%', height='180px'))
run_paste_btn  = w.Button(description='▶️ 実行', button_style='success', layout=w.Layout(width='100px'))
paste_log_out  = w.Output()

def do_gen_prompt(btn):
    req = man_req_box.value.strip()
    if not req:
        with prompt_out:
            clear_output(); print('⚠️ リクエストを入力してください')
        return
    full = SYSTEM_PROMPT + '\n\n【作りたいもの】\n' + req
    prompt_out.clear_output()
    with prompt_out:
        display(HTML('<b>📋 以下をコピーして外部LLMに貼り付けてください：</b>'
                     f'<div class="cad-code" style="margin-top:6px">{full}</div>'))

def do_run_paste(btn):
    code = extract_code(paste_box.value.strip())
    paste_log_out.clear_output()
    with paste_log_out:
        if not code:
            print('⚠️ コードを貼り付けてください'); return
        ok, err = run_code(code)
        if ok:
            print('✅ 実行成功！output/llm_output.step / .stl を確認してください')
        else:
            print(f'❌ エラー:\n{err}')

gen_prompt_btn.on_click(do_gen_prompt)
run_paste_btn.on_click(do_run_paste)

manual_tab = w.VBox([
    w.HTML('<b>作りたいCADモデルを日本語で入力してください</b>'),
    man_req_box, gen_prompt_btn, prompt_out,
    w.HTML('<hr><b>LLMから返ってきたコードを貼り付けて実行</b>'),
    paste_box, run_paste_btn, paste_log_out,
])

# ══════════════════════════════════════════════════════════════
# 📜 会話履歴タブ
# ══════════════════════════════════════════════════════════════
history_out = w.Output()

def refresh_history():
    history_out.clear_output()
    with history_out:
        if not state['history']:
            print('履歴なし'); return
        for i, m in enumerate(state['history']):
            role = '🧑 You' if m['role']=='user' else '🤖 LLM'
            s = m['content'][:140].replace('\n',' ')
            print(f'[{i}] {role}: {s}{"..." if len(m["content"])>140 else ""}')

refresh_history()

hist_tab = w.VBox([w.HTML('<b>会話履歴（APIモード）</b>'), history_out])

# ══════════════════════════════════════════════════════════════
# ダッシュボード組立
# ══════════════════════════════════════════════════════════════
tabs = w.Tab(children=[settings_tab, api_tab, manual_tab, hist_tab])
tabs.set_title(0, '⚙️ API設定')
tabs.set_title(1, '🤖 API 自動生成')
tabs.set_title(2, '📋 Manual')
tabs.set_title(3, '📜 会話履歴')

dashboard = w.VBox([
    w.HTML('<h3 style="margin:4px 0 8px">🔧 Build123d × LLM ダッシュボード</h3>'),
    tabs,
], layout=w.Layout(padding='8px'))

display(dashboard)
print('✅ GUI 起動完了  ─  まず「⚙️ API設定」タブでキーを設定してください')


## (参考) 旧プロンプトテンプレート

APIキーをお持ちでない場合、セル7-Bの `LLM_MODE = 'manual'` を使うと  
同等のプロンプトが自動生成されます。このセクションは参考情報です。


In [15]:
PROMPT_TEMPLATE = """
あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネストは使わない）
  extrude(amount=length)

【配置・パターン】
- Locations((x, y, z)): 指定座標に移動
- PolarLocations(radius, count): 円周上に均等配置
- GridLocations(x_spacing, y_spacing, x_count, y_count): グリッド配置

【よくある間違い・注意事項】
- BuildSketch を入れ子（ネスト）にしない
- 穴はすべて mode=Mode.SUBTRACT で指定する
- fillet はすべての形状が確定した後、最後に適用する

【作りたいもの】
"""

print("📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：")
print("="*60)
print(PROMPT_TEMPLATE)
print("="*60)
print("\n例：【作りたいもの】")
print("外径60mm、内径40mm、長さ100mmのパイプ。両端にR3のフィレットあり。")

📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：

あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネス

## 8. 手動貼り付け実行セル（manualモード用）

`LLM_MODE = 'manual'` でLLMから返ってきたコードをここに貼り付けて実行してください。  
`LLM_MODE = 'api'` の場合はセル7-Bで自動実行されるため、このセルは不要です。


In [17]:
# ============================================================
# ↓ ここにLLMが生成したコードを貼り付けてください
# ============================================================

from build123d import *
import os
os.makedirs("output", exist_ok=True)

# --- LLM生成コードをここに貼る ---
from build123d import *

# ==========================================
# パラメータ定義
# ==========================================
outer_r = 50.0 / 2      # 外径50mmの半径
inner_r = 35.0 / 2      # 内径35mmの半径
branch_len = 100.0      # 長さ各方向100mm
hole_r = 8.0 / 2        # 貫通穴の直径8mmの半径
hole_offset = 75.0      # 穴の中心からの配置位置（必要に応じて変更可能）

with BuildPart() as part:
    # ------------------------------------------
    # 1. クロスパイプジョイントの基本形状作成
    # ------------------------------------------
    # X軸方向のパイプ
    with BuildSketch(Plane.YZ) as sk_x:
        Circle(outer_r)
        Circle(inner_r, mode=Mode.SUBTRACT)
    extrude(amount=branch_len, both=True)
    
    # Y軸方向のパイプ
    with BuildSketch(Plane.XZ) as sk_y:
        Circle(outer_r)
        Circle(inner_r, mode=Mode.SUBTRACT)
    extrude(amount=branch_len, both=True)
    
    # ------------------------------------------
    # 2. 外周から直角方向の穴作成 (4本均等)
    # ------------------------------------------
    # 各パイプの枝に対して、Z軸方向（外周から直角）に直径8mmの穴を貫通させる
    with Locations((hole_offset, 0, 0), (-hole_offset, 0, 0), 
                   (0, hole_offset, 0), (0, -hole_offset, 0)):
        Cylinder(radius=hole_r, height=outer_r * 3, mode=Mode.SUBTRACT)

# ==========================================
# ファイル出力
# ==========================================
export_step(part.part, "output.step")
export_stl(part.part, "output.stl")

# ============================================================
print("✅ 実行完了 / outputフォルダにSTEP・STLが保存されました")

✅ 実行完了 / outputフォルダにSTEP・STLが保存されました


---
## 補足メモ

**Colabでの3Dプレビュー**  
Colab上でのインタラクティブな3D表示は現時点では制限があります。  
STLファイルを出力して、別途Three.jsビューワーや外部ツール（MeshLab、FreeCAD等）で確認してください。

**ローカル環境でのプレビュー**  
VS Code + `ocp_vscode` 拡張機能を使うとリアルタイムで3D確認できます。
```
pip install ocp_vscode
```

**後続パーツとの連携**  
- STLファイル → pygmshメッシャーへ渡す  
- STEPファイル → FreeCADやCalculiXのCAEツールへ渡す

**参考リンク**  
- [Build123d 公式ドキュメント](https://build123d.readthedocs.io/)  
- [Build123d GitHub](https://github.com/gumyr/build123d)